# Import Libraries

In [46]:
import torch # standard basic library for pytorch
import torch.autograd # need Variable class
import torch.nn as nn # need the basic neural net module
import torch.nn.functional as F # imported for the conv1d function

import torch.nn.parameter as Parameter

# Normal Convolution

In [113]:
convFilter = torch.FloatTensor([8.0,2.0]) # use [8,2] for test kernel
convFilter = torch.unsqueeze(convFilter,0) # unsqueeze twice to make filter a 3 dimensional 1x1x2 filter
convFilter = torch.unsqueeze(convFilter,0)
convInput = torch.unsqueeze(torch.autograd.Variable(torch.FloatTensor([3.0,7.0,6.0,4.0])),0) # use [3,7,6,4] as test input
convInput = torch.unsqueeze(convInput,0) #unsqueeze twice to make input 3D 1x1x4 for the Conv1D class
print(convInput)


causal_conv = torch.nn.Conv1d(1,1,2,bias = False) # set bias to false for now since no learning is required yet
causal_conv.weight = torch.nn.Parameter(convFilter) # set weight of conv filter to be what was specified
print(causal_conv.weight)

convOutput = causal_conv(convInput) # convolve input with filter --> this should give non-causal convolution
print(convOutput)

Variable containing:
(0 ,.,.) = 
  3  7  6  4
[torch.FloatTensor of size 1x1x4]

Parameter containing:
(0 ,.,.) = 
  8  2
[torch.FloatTensor of size 1x1x2]

Variable containing:
(0 ,.,.) = 
  38  68  56
[torch.FloatTensor of size 1x1x3]



# Causal Convolution

In [112]:
class CausalConv(torch.nn.Conv1d):
    def __init__(self,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False):
        super(CausalConv,self).__init__()
        